In [1]:
import csv
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AdamW
import random

import numpy as np
from datasets import load_metric

/home/henry/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model loading
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [9]:
# data loading, might take a bit
import math
SPLIT = 0.8

data = csv.reader(open("./clues_tags.csv", 'r'))
data = [*data]
random.shuffle(data)
_, _, text, _, labels = list(zip(*data))

unique_labels = list(set(labels))
unique_labels.sort()

def label_to_index(l):
    return unique_labels.index(l)

labels = list(map(label_to_index, labels))

train_texts, test_texts = list(text[:math.floor(SPLIT*len(text))]), list(text[math.floor(SPLIT*len(text)):])
train_labels, test_labels = list(labels[:math.floor(SPLIT*len(text))]), list(labels[math.floor(SPLIT*len(text)):])

from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

# prepare 10000 token vectors
train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')
val_encodings = tokenizer(val_texts, padding=True, truncation=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')

In [10]:
# creating pytorch dataset interfaces
import torch

class CrossWordDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, i):
        item = {key: torch.tensor(val[i]) for key, val in self.encodings.items()}
        print(item)
        item['labels'] = torch.tensor(self.labels[i])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CrossWordDataset(train_encodings, train_labels)
val_dataset = CrossWordDataset(val_encodings, val_labels)
test_dataset = CrossWordDataset(test_encodings, test_labels)

In [11]:
# download model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=len(unique_labels), problem_type= "single_label_classification")

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/henry/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
   

In [13]:
# train model

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    )
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    )

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/henry/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 640
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 240
/tmp/ipykernel_72364/2052539018.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() 

{'input_ids': tensor([  101, 13063,  6533,  3946, 17310,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([  101, 20979,  1104, 22984,  1757,   117,  3229,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([ 101,  139, 3484, 1566,  112,  188, 4074,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([ 101, 4614, 2186,  102,    0,    0,    0,    0,    0,    0,    0,   

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.036823,0.768750
2,No log,1.120515,0.787500
3,No log,1.246264,0.756250


{'input_ids': tensor([ 101, 5823, 1103, 1834, 3979,  102,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([  101, 17228,  1279,  1120,  1313,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([ 101, 8442, 1183, 9877, 7253, 1197,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([  101,   107,  1109,   168,   168,  1104, 20451, 22763,   107,   113,
         2424, 

{'input_ids': tensor([ 101, 4209, 7201,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([  101, 16752, 19482,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([  101, 15463, 19515,  1818,  1830,  1106, 17043,  5345,  1757,   102,
            0,     0,     0,     0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([ 101, 5549,  112,  188, 1148, 1676,  102,    0,    0,    0,    0,   

...

{'input_ids': tensor([  101, 14098,  1246,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([  101,  3131,  1137, 26593,   102,     0,     0,     0,     0,     0,
            0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([ 101, 4304, 1736,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([  101, 17077,  1174,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 0, 0, 0, 0, 0,

{'input_ids': tensor([ 101,  107, 2957, 2563, 6060,  107, 2351,  102,    0,    0,    0,    0,
           0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0])}
{'input_ids': tensor([ 101,  107,  143, 7264, 1777, 3101,  117,  107,  174,  119,  176,  119,
         102]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
{'input_ids': tensor([  101, 16482,  3667,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([ 101, 1124, 2086, 1594,  102,    0,    0,    0,    0,    0,    0,    0,
           0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids

{'input_ids': tensor([  101, 24187,  1106,   170, 21950,   117,  7269,  1193,   102,     0,
            0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])}
{'input_ids': tensor([  101,  1556,   107,  1109,   117,   107,  2594, 20825,  1326,   102,
            0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0])}
{'input_ids': tensor([  101, 10736,   170,  1363, 15772,  1289,   102,     0,     0,     0,
            0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([  101,  9198, 14846,  1162,   102,     0,     0,     0,     0,     0,
            0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=240, training_loss=0.49631354014078777, metrics={'train_runtime': 311.9465, 'train_samples_per_second': 6.155, 'train_steps_per_second': 0.769, 'total_flos': 16775438192640.0, 'train_loss': 0.49631354014078777, 'epoch': 3.0})